In [ ]:
##############################################
##  git pjm9889에 있는 csv자료를 클릭하고 우측 상단의 raw를 클릭해서 나타나는 상단의 url을 복사해서 path로 사용할수 있음.
###############################################
import pandas as pd
path='https://raw.githubusercontent.com/pjm9889/kpmg/refs/heads/main/1125-1129/%EB%8F%84%EC%8B%9C%EB%9D%BD.csv'
df=pd.read_csv(path)
df.head(1)

,title,link,image,lprice,hprice,mallName,productId,productType,brand,maker,category1,category2,category3,category4
0,다소다 도시락통 <b>직장인</b> 스텐도시락 샐러드 전자레인지 보온용기 소풍 수능...,https://smartstore.naver.com/main/products/604...,https://shopping-phinf.pstatic.net/main_835846...,17900,NaN,다소다 공식스토어,83584640285,2,NaN,NaN,생활/건강,주방용품,보관/밀폐용기,도시락통/찬합


In [ ]:
# 'title'열을 'title.txt' 파일로 저장 (한글 인코딩)
with open('title.txt', 'w', encoding='utf-8') as f:
    for title in df['title']:
        f.write(str(title) + '\n')

In [ ]:
#######################################
## 랭체인 로더기
## https://wikidocs.net/231578
###########################################
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader('/content/title.txt',encoding='utf-8')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=100,
    encoding_name='cl100k_base'
)
#'다소다 도시락통 <b>직장인</b> 스텐도시락 샐러드 전자레인지 보온용기 소풍 수능 런치박스'
texts = text_splitter.split_text(data[0].page_content)
len(texts)

64

In [103]:
# !pip install chromadb

In [ ]:
#######################3
## 임베딩 모델 초기화
##########################

embeddings_model = OpenAIEmbeddings()
db = Chroma.from_texts(
    texts,
    embeddings_model,
    collection_name = 'history',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db

In [ ]:

query = '눈이 오는 날에 직장 필수템은?'
docs = db.similarity_search(query)
docs[0]

Document(metadata={}, page_content='점심 사각 보온')

In [ ]:
#############################################
## GPT 유료버전없이 API키를 이용하여서 질문을 하고자 함.
## 데이터베이스 구축했다면 아래 내용을 db에 저장하여서 지식 db화 하고 추후, 문제지의 형태로 질문을 만들수도 있음.
## 히스토리 기능을 이용해서 gpt 유료버전에게 질문
############################################
from IPython.display import display
from IPython.display import Markdown

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import os
openai_api_key = 'openAPI***************************'
os.environ['OPENAI_API_KEY'] = openai_api_key


memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)

msg = '크로마 db하고 fasii 차이점, 자세하고 쉽게'

result = conversation.run(msg)
display(Markdown(result))

크로마 DB와 FASII는 서로 다른 분야에서 사용되는 두 가지 기술로, 각각의 목적과 기능이 다릅니다. 여기서 각각의 차이점을 자세하고 쉽게 설명해드릴게요.

1. **크로마 DB (Chroma DB):**
   - **목적 및 사용 사례**: 크로마 DB는 주로 벡터 데이터베이스로 사용됩니다. 벡터 데이터베이스는 주로 머신러닝과 인공지능 분야에서 사용되며, 특히 딥러닝 모델에서 생성된 임베딩을 저장하고 검색하는 데 유용합니다.
   - **특징**: 크로마 DB는 고차원 벡터를 효율적으로 저장하고, 이러한 벡터들 간의 유사성을 빠르게 검색하는 기능을 제공합니다. 이를 통해 이미지, 텍스트, 음성 데이터 등 다양한 유형의 데이터를 처리할 수 있습니다.
   - **장점**: 대규모 데이터셋을 다루는 경우에도 빠른 검색이 가능하며, 머신러닝 모델의 성능을 향상시키는 데 도움을 줄 수 있습니다.

2. **FASII (Flexible Advanced Storage Integration Infrastructure):**
   - **목적 및 사용 사례**: FASII는 주로 저장 장치의 인프라를 관리하고 최적화하는 데 중점을 둡니다. 대규모 데이터 센터나 엔터프라이즈 환경에서 복잡한 저장 요구를 처리하기 위해 설계되었습니다.
   - **특징**: FASII는 다양한 스토리지 기술과 프로토콜을 지원하며, 유연한 통합 및 확장이 가능합니다. 이를 통해 기업은 그들의 데이터 관리 전략을 보다 효율적으로 운영할 수 있습니다.
   - **장점**: 스토리지 자원의 활용도를 극대화할 수 있으며, 다양한 저장 장치와의 통합이 용이하여 비용 효율성을 제공합니다.

이 두 기술은 각각의 특수한 목적과 사용 사례에 맞게 설계되어 있으며, 데이터 처리나 저장의 특정 요구 사항에 따라 선택적으로 사용될 수 있습니다.

In [108]:
msg = 'from langchain_openai import OpenAIEmbeddings 에서 임베딩쿼리의 값을 1개만 리턴받고자 할때'

result = conversation.run(msg)
display(Markdown(result))

`langchain_openai` 라이브러리에서 `OpenAIEmbeddings`를 사용하여 임베딩 쿼리를 수행할 때, 결과에서 하나의 임베딩 값만을 반환받고자 할 경우, 일반적으로 쿼리 결과에서 첫 번째 항목만 선택하도록 코드를 작성해야 합니다. 일반적인 Python 코드 예제를 통해 설명해드릴게요:

```python
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings()

# 쿼리를 통해 임베딩 값 얻기
query = "This is a sample query."
embedding_result = embeddings.embed(query)

# 결과에서 첫 번째 임베딩 값만 선택
first_embedding = embedding_result[0]

# 첫 번째 임베딩 값 출력
print(first_embedding)
```

이 코드에서는 `embed()` 메서드를 통해 쿼리의 임베딩을 얻고, `embedding_result` 리스트의 첫 번째 요소를 선택하여 `first_embedding`에 저장합니다. 이 방법을 통해 필요한 하나의 임베딩 값을 얻을 수 있습니다. 사용하려는 라이브러리의 버전에 따라 세부적인 사용법이나 메서드 이름이 다를 수 있으니, 최신 문서를 참고하여 사용하는 것이 좋습니다.

In [109]:
msg = '코드 없이 개념적 설명만, 필요, 통계적 기법을 넣어서 각각의 장단점을 넣어서'

result = conversation.run(msg)
display(Markdown(result))

임베딩 쿼리에서 하나의 값만 반환받는 개념을 통계적 기법과 함께 설명드릴게요.

1. **개념적 설명**:
   - 임베딩은 주어진 텍스트 또는 데이터를 고차원 벡터 공간에 매핑하는 과정입니다. 이를 통해 기계는 자연어 같은 복잡한 데이터를 수학적으로 처리할 수 있게 됩니다.
   - 여러 개의 임베딩 값 중 하나만 선택한다는 것은, 특정 쿼리에 대해 가장 관련성이 높은 임베딩이나 특정 기준에 따라 선택된 단일 벡터를 의미합니다.

2. **통계적 기법 사용**:
   - **평균**: 여러 임베딩 값의 평균을 구해 대표값으로 사용하는 방법이 있습니다. 이는 전체 데이터의 일반적인 경향을 파악할 수 있게 해주지만, 개별 데이터의 세부적인 차이를 놓칠 수 있습니다.
   - **중앙값**: 데이터의 중앙값을 사용하여 극단적인 값의 영향을 줄일 수 있습니다. 이는 불균형한 데이터 분포에서 유용할 수 있습니다.
   - **최대값 또는 최소값**: 임베딩 벡터에서 특정 차원의 최대값이나 최소값을 선택하여 특정 특성을 강조할 수 있습니다.

3. **장단점**:
   - **장점**:
     - 하나의 임베딩 값을 사용하면 복잡성을 줄일 수 있으며, 처리 속도가 빨라집니다.
     - 특정 기준에 따라 선택된 임베딩은 특정한 특성을 강조하거나 분석할 때 유용합니다.

   - **단점**:
     - 하나의 값만 사용하면 데이터의 다양한 측면을 반영하기 어려울 수 있습니다.
     - 선택된 임베딩 값이 전체 데이터의 복잡성을 충분히 반영하지 못할 위험이 있습니다.

결론적으로, 임베딩 쿼리에서 하나의 값을 선택하는 방법은 데이터의 특성과 분석 목적에 따라 달라질 수 있습니다. 각 방법의 장단점을 고려하여 적절한 기법을 선택하는 것이 중요합니다.

In [111]:
msg = '이러한 벡터 임베딩의 역사는'

result = conversation.run(msg)
display(Markdown(result))

벡터 임베딩의 역사는 자연어 처리(NLP)와 정보 검색 분야에서 발전해온 여러 기술들과 깊이 얽혀 있습니다. 이 역사적인 발전을 간단히 정리해보겠습니다.

1. **초기 통계적 방법 (1970s-1980s):**
   - 벡터 공간 모델(Vector Space Model)과 TF-IDF(Term Frequency-Inverse Document Frequency) 같은 초창기 기법들이 문서를 수치적으로 표현하기 시작했습니다. 이러한 방법들은 주로 단어의 빈도를 기반으로 문서의 특징을 벡터로 나타냈습니다.

2. **잠재 의미 분석 (LSA, 1990s):**
   - 잠재 의미 분석(Latent Semantic Analysis)은 대규모 문서 집합에서 단어와 문서의 관계를 파악하기 위해 행렬 분해를 사용하여 단어를 벡터로 표현하는 방법입니다. LSA는 단어의 의미를 통계적으로 분석하여 숨겨진 의미 구조를 찾으려고 했습니다.

3. **신경망 기반 임베딩 (2000s):**
   - 신경망을 사용한 방법들이 등장하면서, 단어 의미를 더 세밀하게 모델링할 수 있게 되었습니다. 특히, 워드 임베딩(Word Embedding)이 널리 사용되기 시작했습니다.

4. **Word2Vec (2013):**
   - 구글에서 개발한 Word2Vec은 단어를 벡터로 임베딩하는 방법으로 큰 주목을 받았습니다. Word2Vec은 단어 사이의 유사성을 벡터 공간에서 유의미하게 모델링할 수 있도록 하였으며, "king" - "man" + "woman" ≈ "queen" 같은 벡터 연산이 가능하게 했습니다.

5. **GloVe 및 FastText (2014-2016):**
   - GloVe(Global Vectors for Word Representation)는 Stanford에서 개발한 단어 임베딩 기법으로, 전체 코퍼스의 통계적 정보를 사용하여 단어 벡터를 학습합니다.
   - FastText는 Facebook AI Research에서 개발했으며, 단어를 문자 n-그램의 조합으로 처리하여, 드물거나 새로운 단어에 대해서도 의미 있는 임베딩을 생성할 수 있도록 합니다.

6. **컨텍스트 기반 임베딩 (ELMo, BERT, 이후, 2018-현재):**
   - ELMo(Embeddings from Language Models)와 BERT(Bidirectional Encoder Representations from Transformers) 같은 모델들은 문맥을 고려한 임베딩을 제공합니다. 이는 단어의 의미가 문맥에 따라 달라질 수 있음을 반영합니다.
   - 이러한 모델들은 대규모 데이터와 강력한 컴퓨팅 파워를 활용하여 다양한 NLP 작업에서 뛰어난 성능을 보여주고 있습니다.

이와 같은 발전들은 자연어 처리 분야에서 컴퓨터가 인간 언어를 더욱 효과적으로 이해하고 처리할 수 있도록 기여해 왔습니다.

In [112]:
msg = '''이해도 테스트 문제

문제1. 크로마 db하고 fasii 차이점, 자세하고 쉽게
문제2. from langchain_openai import OpenAIEmbeddings 에서 임베딩쿼리의 값을 1개만 리턴받고자 할때
문제3. 통계적 기법을 넣어서 각각의 장단점을 넣어서
문제4. 벡터임베딩 역사 '''



result = conversation.run(msg)
display(Markdown(result))

여기 각 문제에 대한 이해도를 테스트하기 위한 문제를 준비해보았습니다.

**문제 1: 크로마 DB와 FASII의 차이점**
- 크로마 DB와 FASII는 각각 어떤 목적으로 사용되며, 어떤 특징을 가지고 있는지 설명하세요.
- 벡터 데이터베이스와 저장 장치 인프라의 차이를 중심으로 두 기술의 장단점을 비교해보세요.

**문제 2: 임베딩 쿼리에서 하나의 값을 반환받는 방법**
- `langchain_openai` 라이브러리의 `OpenAIEmbeddings`에서 임베딩 쿼리를 수행할 때, 결과에서 하나의 임베딩 값만을 선택하는 개념을 설명하세요.
- 이렇게 하는 것이 왜 유용할 수 있는지 예를 들어 설명해보세요.

**문제 3: 통계적 기법을 활용한 임베딩 선택의 장단점**
- 임베딩 결과에서 하나의 값을 선택하기 위해 사용할 수 있는 통계적 기법에는 어떤 것들이 있는지 열거하고, 각 기법의 장점과 단점을 설명하세요.
- 이러한 기법들이 결과에 미치는 영향을 어떻게 평가할 수 있을지 논의해보세요.

**문제 4: 벡터 임베딩의 역사**
- 벡터 임베딩의 발전 과정을 시대별로 나누어 설명하세요.
- 각 단계에서 중요한 기술적 발전과 그 영향에 대해 서술해보세요.
- 최근의 컨텍스트 기반 임베딩 기술이 이전 기술들과 어떻게 다른지 설명하세요.

각 문제에 대한 답변을 통해 벡터 임베딩과 관련된 기술들을 얼마나 깊이 이해하고 있는지 평가할 수 있습니다.

In [113]:
msg = ''' PM 면접시에 나올수 있는 예상질문지 뽑아줘. 답도 같이 알려줘 (초급Pm과 중급PM모두)

문제1. 크로마 db하고 fasii 차이점, 자세하고 쉽게
문제2. from langchain_openai import OpenAIEmbeddings 에서 임베딩쿼리의 값을 1개만 리턴받고자 할때
문제3. 통계적 기법을 넣어서 각각의 장단점을 넣어서
문제4. 벡터임베딩 역사 '''



result = conversation.run(msg)
display(Markdown(result))

아래는 초급 및 중급 프로젝트 매니저(PM) 면접에서 나올 수 있는 예상 질문과 답변입니다. 각 질문은 데이터베이스 기술, 임베딩 처리 방법, 통계적 기법, 그리고 벡터 임베딩의 역사에 대한 이해를 평가할 수 있습니다.

### 문제 1: 크로마 DB와 FASII의 차이점
#### 예상 질문
- 크로마 DB와 FASII는 각각 어떤 기술이며, 어떤 차이점이 있습니까? 두 기술의 장단점을 설명해주세요.

#### 답변
- **크로마 DB**는 벡터 데이터베이스로, 주로 머신러닝과 AI 분야에서 임베딩을 저장하고 검색하는 데 사용됩니다. 대규모 데이터셋에서 빠른 검색이 가능하도록 설계되어 있습니다.
- **FASII**는 저장 장치 인프라를 관리하고 최적화하는 시스템으로, 다양한 스토리지 기술과 프로토콜을 지원하여 기업의 데이터 관리 전략을 효율적으로 운영할 수 있게 해줍니다.
- **장점**: 크로마 DB는 대규모 데이터셋에서의 빠른 검색 기능이 뛰어나고, FASII는 스토리지 자원의 활용도를 극대화하고 비용 효율성을 제공합니다.
- **단점**: 크로마 DB는 벡터 데이터에 특화되어 있어 다른 유형의 데이터 처리에는 제한적일 수 있으며, FASII는 인프라 관리에 초점을 맞추다 보니 벡터 데이터 처리에는 적합하지 않을 수 있습니다.

### 문제 2: 임베딩 쿼리에서 하나의 값을 반환받는 방법
#### 예상 질문
- `langchain_openai` 라이브러리에서 임베딩 쿼리를 수행할 때, 결과에서 하나의 임베딩 값만 선택하는 방법을 설명해 주세요.

#### 답변
- 임베딩 쿼리를 통해 얻어진 복수의 벡터 중 하나를 선택하는 것은 특정 목적에 맞게 데이터를 단순화하는 과정입니다. 예를 들어, 대량의 임베딩 중 가장 관련성이 높은 또는 특정 기준에 맞춘 하나의 벡터를 선택할 수 있습니다.
- 이렇게 하는 것은 데이터의 복잡성을 줄이고, 처리 속도를 향상시키며 특정 특성을 강조할 수 있는 장점이 있습니다.

### 문제 3: 통계적 기법을 활용한 임베딩 선택의 장단점
#### 예상 질문
- 임베딩 결과에서 하나의 값을 선택하기 위해 사용할 수 있는 통계적 기법과 각 기법의 장단점을 설명해주세요.

#### 답변
- **평균**: 모든 임베딩의 평균을 사용하여 일반적인 경향을 파악할 수 있지만, 개별 데이터의 차이를 놓칠 수 있습니다.
- **중앙값**: 중앙값을 사용하면 극단적인 값의 영향을 줄일 수 있으며, 불균형한 데이터 분포에서 유용합니다.
- **최대값/최소값**: 특정 차원의 최대값이나 최소값을 선택하여 특정 특성을 강조할 수 있습니다. 하지만, 이는 전체 맥락을 반영하지 않을 수 있습니다.

### 문제 4: 벡터 임베딩의 역사
#### 예상 질문
- 벡터 임베딩의 발전 과정을 설명하고, 최근의 컨텍스트 기반 임베딩 기술이 이전 기술들과 어떻게 다른지 설명해 주세요.

#### 답변
- 벡터 임베딩은 초기에 TF-IDF와 같은 통계적 방법으로 시작하여, LSA를 통해 의미 구조를 분석했습니다.
- Word2Vec, GloVe, FastText 등의 신경망 기반 임베딩은 단어의 유사성을 벡터로 표현할 수 있는 강력한 방법을 제공했습니다.
- 최근의 ELMo, BERT와 같은 컨텍스트 기반 임베딩은 문맥을 고려하여 단어의 의미를 더 정확하게 모델링합니다. 이는 이전의 고정된 임베딩과 달리, 문맥에 따른 의미 변화를 반영할 수 있습니다.

이와 같은 질문들은 면접에서 PM의 기술적 이해와 프로젝트 관리 능력을 평가하는 데 도움을 줄 수 있습니다. 각 답변은 면접자의 이해도를 보여주며, 구체적인 사례를 통해 답변을 보강하면 더욱 좋습니다.

In [114]:
msg = ''' 질문지 먼저 나오고, 답변지를 초급PM, 중급PM으로 구별해서

문제1. 크로마 db하고 fasii 차이점, 자세하고 쉽게
문제2. from langchain_openai import OpenAIEmbeddings 에서 임베딩쿼리의 값을 1개만 리턴받고자 할때
문제3. 통계적 기법을 넣어서 각각의 장단점을 넣어서
문제4. 벡터임베딩 역사 '''



result = conversation.run(msg)
display(Markdown(result))

### 질문지

1. **크로마 DB와 FASII의 차이점**
   - 크로마 DB와 FASII는 각각 어떤 기술이며, 어떤 차이점이 있습니까? 두 기술의 장단점을 설명해주세요.

2. **임베딩 쿼리에서 하나의 값을 반환받는 방법**
   - `langchain_openai` 라이브러리에서 임베딩 쿼리를 수행할 때, 결과에서 하나의 임베딩 값만 선택하는 방법을 설명해 주세요.

3. **통계적 기법을 활용한 임베딩 선택의 장단점**
   - 임베딩 결과에서 하나의 값을 선택하기 위해 사용할 수 있는 통계적 기법과 각 기법의 장단점을 설명해주세요.

4. **벡터 임베딩의 역사**
   - 벡터 임베딩의 발전 과정을 설명하고, 최근의 컨텍스트 기반 임베딩 기술이 이전 기술들과 어떻게 다른지 설명해 주세요.

---

### 답변지

#### 초급 PM

1. **크로마 DB와 FASII의 차이점**
   - 크로마 DB는 벡터 데이터베이스로, AI와 머신러닝의 임베딩 데이터를 저장하고 검색하는 데 사용됩니다. FASII는 저장 장치 인프라를 관리하는 기술로, 대규모 데이터 센터에서 스토리지 자원을 효율적으로 운영하는 데 중점을 둡니다.
   - 크로마 DB는 빠른 벡터 검색이 장점이고, FASII는 다양한 스토리지 통합이 장점입니다.

2. **임베딩 쿼리에서 하나의 값을 반환받는 방법**
   - 여러 임베딩 값 중 하나를 선택할 때는 주로 가장 관련성이 높은 값을 선택합니다. 이렇게 하면 복잡성을 줄이고, 빠른 처리가 가능합니다.

3. **통계적 기법을 활용한 임베딩 선택의 장단점**
   - 평균을 사용하면 데이터의 일반적인 경향을 파악할 수 있습니다. 그러나 개별 차이를 놓칠 수 있습니다. 중앙값은 극단값의 영향을 줄여줍니다.

4. **벡터 임베딩의 역사**
   - TF-IDF에서 시작하여, Word2Vec, GloVe 같은 방법들이 발전했습니다. 최근에는 문맥을 고려하는 BERT가 주목받고 있습니다.

#### 중급 PM

1. **크로마 DB와 FASII의 차이점**
   - 크로마 DB는 벡터 임베딩 데이터의 저장과 검색 최적화에 중점을 두어, 머신러닝 모델에서 고차원 벡터를 효과적으로 처리합니다. 반면, FASII는 다양한 스토리지 기술을 통합하여 인프라의 유연성을 제공합니다.
   - 크로마 DB는 대량의 벡터 데이터 검색에서 성능을 발휘하고, FASII는 다양한 저장 요구를 처리하여 비용 효율성을 제공합니다.

2. **임베딩 쿼리에서 하나의 값을 반환받는 방법**
   - 임베딩 결과에서 단일 값을 선택하는 것은 데이터의 특정 측면을 강조하거나, 연산 효율성을 위해 데이터를 단순화하는 전략입니다. 이는 특정 임베딩이 분석의 핵심이 되도록 합니다.

3. **통계적 기법을 활용한 임베딩 선택의 장단점**
   - 평균은 전체 데이터의 경향을 나타내지만, 데이터의 다양성을 간과할 수 있습니다. 중앙값은 극단적인 값의 영향을 줄이며, 최대/최소값은 특정 특성을 강조합니다. 그러나 이는 전체 문맥을 반영하지 않을 수 있습니다.

4. **벡터 임베딩의 역사**
   - 초기의 TF-IDF와 LSA는 기본적인 통계적 방법이었고, Word2Vec과 GloVe 등은 신경망 기반의 벡터 임베딩을 가능하게 했습니다. 최근의 ELMo와 BERT는 문맥을 반영한 임베딩으로, 문장의 의미를 더 정확하게 모델링합니다. 이는 이전의 고정 임베딩과 차별화되며, 문맥에 따른 의미 변화를 효과적으로 포착합니다.

In [116]:
msg = ' 면접 구어체로'    # 잘안나옴... 변경하여 보세요.

result = conversation.run(msg)
display(Markdown(result))

### 질문지

1. **크로마 DB와 FASII의 차이점**
   - 크로마 DB랑 FASII는 각각 어떤 기술이고, 뭐가 다른가요? 두 기술의 장단점을 쉽게 설명해 주세요.

2. **임베딩 쿼리에서 하나의 값을 반환받는 방법**
   - `langchain_openai` 라이브러리에서 임베딩 쿼리를 할 때, 결과에서 하나의 임베딩 값만 선택하는 방법을 설명해 주세요.

3. **통계적 기법을 활용한 임베딩 선택의 장단점**
   - 임베딩 결과에서 하나의 값을 고를 때 쓸 수 있는 통계적 기법과 그 장단점을 설명해 주세요.

4. **벡터 임베딩의 역사**
   - 벡터 임베딩이 어떻게 발전해 왔는지, 최근의 컨텍스트 기반 임베딩 기술이 이전 기술과 어떻게 다른지 설명해 주세요.

---

### 답변지

#### 초급 PM

1. **크로마 DB와 FASII의 차이점**
   - 크로마 DB는 벡터 데이터베이스로 AI랑 머신러닝에서 임베딩 데이터를 저장하고 빨리 찾는 데 쓰여요. FASII는 저장 장치 인프라를 관리하는 거라서, 큰 데이터 센터에서 스토리지 자원을 잘 운영하는 데 도움을 줘요.
   - 크로마 DB는 벡터 검색이 빠르고, FASII는 여러 스토리지를 쉽게 통합할 수 있어요.

2. **임베딩 쿼리에서 하나의 값을 반환받는 방법**
   - 여러 임베딩 값 중에서 하나를 고를 땐, 가장 관련 있는 값을 뽑아내요. 이렇게 하면 데이터가 더 간단해지고, 처리 속도가 빨라져요.

3. **통계적 기법을 활용한 임베딩 선택의 장단점**
   - 평균을 쓰면 전체 데이터의 경향을 알 수 있지만, 개별 차이는 잘 못 볼 수도 있어요. 중앙값은 극단적인 값의 영향을 줄여주니까 좋죠.

4. **벡터 임베딩의 역사**
   - 처음엔 TF-IDF 같은 기본적인 방법으로 시작해서, Word2Vec, GloVe 같은 방법들이 나왔어요. 요즘엔 문맥을 고려하는 BERT 같은 게 많이 쓰여요.

#### 중급 PM

1. **크로마 DB와 FASII의 차이점**
   - 크로마 DB는 벡터 임베딩 데이터를 다루는 데 최적화되어 있어서 머신러닝 모델에서 벡터를 잘 처리해요. 반면에 FASII는 다양한 스토리지 기술을 통합해서 인프라의 유연성을 높여줘요.
   - 크로마 DB는 대량의 벡터 데이터를 빠르게 검색하는 데 강점이 있고, FASII는 다양한 스토리지 요구를 처리하면서 비용도 효율적으로 관리해요.

2. **임베딩 쿼리에서 하나의 값을 반환받는 방법**
   - 임베딩 결과에서 하나만 고르는 건 데이터를 단순화하고 연산을 더 효율적으로 하기 위한 전략이에요. 특정 임베딩이 분석의 핵심이 되도록 하는 거죠.

3. **통계적 기법을 활용한 임베딩 선택의 장단점**
   - 평균은 전체적인 경향을 보여주지만, 데이터의 다양성은 놓칠 수 있어요. 중앙값은 극단값의 영향을 줄여주고, 최대/최소값은 특정 특성을 강조하는 데 쓸 수 있어요. 다만, 전체 문맥을 반영하진 못할 수도 있죠.

4. **벡터 임베딩의 역사**
   - 초기엔 TF-IDF와 LSA 같은 방법들이 있었고, 그 후엔 Word2Vec과 GloVe 같은 신경망 기반 임베딩이 나왔어요. 최근엔 ELMo와 BERT가 문장의 의미를 더 정확하게 모델링하는데, 문맥을 반영해서 이전의 고정된 방식과는 확실히 다르죠. 이게 문맥에 따라 의미가 변하는 걸 잘 잡아내요.